In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import cv2

In [ ]:
batch_size = 
image_width = 
image_height = 
image_size = 



no_grid = 
no_classes = 
no_boxes_per_cell = 

#Scales
class_scale = 
confidence_obj_scale = 
confidence_noobj_scale = 

def conv_layers_typ1(x):
    
    #Repeat Convolutional layer 1
    x1 = tf.layers.conv2d(x, 256,1)

    #Repeat convolutional layer 2
    x2 = tf.layers.conv2d(x1, 512, 3)
    
    return x2


def conv_layers_typ2(x):
    
    #Repeat convolutional layer 1
    x1 = tf.layers.conv2d(x, 512, 1)
    
    #Repeat convolutional layer 2
    
    x2 = tf.layers.conv2d(x1, 1024, 3)
    
    return x2


def yolo_model():
    
    with tf.variable_scope('yolo_model'):
        
        #Convolutional layer 1
        x1 = tf.layers.conv2d(x,64,7,2,'same')

        #Max pool layer 1
        x2 = tf.layers.max_pooling2d(x1,2,2)
        
        #Convolutional layer 2
        x3 = tf.layers.conv2d(x2, 192, 3)
        
        #Max pool layer 2
        x4 = tf.layers.max_pooling2d(x3,2,2)
        
        #Convolutional layer 3
        x5 = tf.layers.conv2d(x4, 128,1)
        
        #Convolutional layer 4
        x6 = tf.layers.conv2d(x5, 256, 3)
        
        #Convolutional layer 5
        x7 = tf.layers.conv2d(x6, 256, 1)
        
        #Convolutional layer 6
        x8 = tf.layers.conv2d(x7, 512, 3)
        
        #Max pool layer 3
        x9 = tf.layers.max_pooling2d(x8, 2,2)
        
        #Convolutional layers 7,8
        x10 = conv_layers_typ1(x9)
        
        #Convolutional layers 9,10
        x11 = conv_layers_typ1(x10)
        
        #Convolutional layers 11,12
        x12 = conv_layers_typ1(x11)
        
        #Convolutional layers 13,14
        x13 = conv_layers_typ1(x12)
        
        #Convolutional layer 15
        x14 = tf.layers.conv2d(x13, 512, 1)
        
        #Convolutional layer 16
        x15 = tf.layers.conv2d(x14, 1024, 3)
        
        #Max pool layer 4
        x16 = tf.layers.max_pooling2d(x15, 2,2)
        
        #Convolutional layer 17, 18
        x17 = conv_layers_typ2(x16)
        
        #Convolutional layer 19, 20
        x18 = conv_layers_typ2(x17)
        
        #Convolutional layer 21
        x19 = tf.layers.conv2d(x18, 1024, 3)
        
        #Convolutional layer 22
        x20 = tf.layers.conv2d(x19, 1024, 3, 2)
        
        #Convolutional layer 23
        x21 = tf.layers.conv2d(x20, 1024, 3)
        
        #Convolutional layer 24
        x22 = tf.layers.conv2d(x21, 1024, 3)
        
        #Fully connected layer 1
        flat1 = tf.contrib.layers.flatten(x22)
        x23 = tf.layers.dense(flat1, 512)
        
        #Fully connected layer 2
        flat2 = tf.contrib.layers.flatten(x23)
        x24 = tf.layers.dense(flat2, 4096)
        
        #Reshaping the output of the last layer to the size (batch size, S,S,(5*B+C))
        out = tf.reshape(x24, (-1, 7,7,30))
        
        return out
    
def calc_iou_single(label_box, predict_box):
    
    #TODO: convert the relative x y with respect to grid cell to absolute x y
    
    #Calculating the actual label box upper left x,y and lower right x,y
    lxa = label_box[0]
    lya = label_box[1]
    lxb = lxa + label_box[2]*image_width
    lyb = lya + label_box[3]*image_height
    
    #Calculating the predicted box upper left x,y and lower right x,y
    pxa = predict_box[0]
    pya = predict_box[1]
    pxb = pxa + label_box[2]*image_width
    pyb = pya + label_box[3]*image_height
    
    #Calculating the intersection box upper left x,y and lower right x,y
    ixa = np.max(lxa, pxa)
    iya = np.max(lya, pya)
    ixb = np.min(lxb, pxb)
    iyb = np.min(lyb, pyb)
    
    #Calculating the area of the intersection box
    iArea = (ixb - ixa + 1)*(iyb - iya + 1)
    
    #Calculating the area of label box and predicted box
    lArea = (lxb - lxa +1)*(lxb - lxa + 1)
    pArea = (pxb - pxa + 1)*(pyb - pyb + 1)
    
    
    #Calculating and returning IOU
    iou = iArea/(lArea + pArea - iArea)
    
def cal_iou(predict_boxes, label_boxes):
    """
    Calculates IOU in multidimension
    
    Input:
    ------
    predict_boxes (batch_size, no_grid, no_grid, boxes_per_cell, 4): predicted boxes (corrected)
    label_boxes (batch_size, no_grid, no_grid, boxes_per_cell, 4): label boxes
    
    Returns:
    --------
    IOU (batch_size, no_grid, no_grid, boxes_per_cell) : multidimensional IOU (truncated between 0 and 1)
    """
    #Calculating the predicted box upper left x,y and lower right x,y
    boxes1 = tf.stack([predict_boxes[:,:,:,:,0],
                       predict_boxes[:,:,:,:,1],
                       (predict_boxes[:,:,:,:,0] + predict_boxes[:,:,:,:,2]),
                       (predict_boxes[:,:,:,:,1] + predict_boxes[:,:,:,:,3])])
    boxes1 = tf.transpose(boxes1, [1,2,3,4,0])
    
    #Calculating the actual label box upper left x,y and lower right x,y
    boxes2 = tf.stack([label_boxes[:,:,:,:,0],
                       label_boxes[:,:,:,:,1],
                       (label_boxes[:,:,:,:,0] + label_boxes[:,:,:,:,2]),
                       (label_boxes[:,:,:,:,1] + label_boxes[:,:,:,:,3])])
    boxes2 = tf.transpose(boxes2, [1,2,3,4,0])
    
    #Calculating the intersection box upper left x,y and lower right x,y
    ul = tf.maximum(boxes1[:,:,:,:,:2], boxes2[:,:,:,:,:2])
    lr = tf.minimum(boxes1[:,:,:,:,2:], boxes2[:,:,:,:,2:])
    
    #Calculating the area of the intersection box
    idiff = tf.maximum(0.0, lr - ul)
    iArea = idiff[:,:,:,:,0] * idiff[:,:,:,:,1]
    
    #Calculating the area of label box and predicted box
    lArea = (boxes2[:,:,:,:,2] - boxes2[:,:,:,:,0])*(boxes2[:,:,:,:,3] - boxes2[:,:,:,:,1])
    pArea = (boxes1[:,:,:,:,2] - boxes1[:,:,:,:,0])*(boxes1[:,:,:,:,3] - boxes1[:,:,:,:,1])
    
    
    #Calculating union area
    uArea = tf.maximum(lArea + pArea - iArea, 1e-10)
    
    #Clipping and returning the IOU
    return tf.clip_by_value(iArea / uArea . 0.0, 1.0)
    
    
def loss(prediction, labels):
    
    """
    #Dimensions of labels:
    Labels will be of dimension [batch_size, no_grid, no_grid, 5 (1st dimension would be confidence (1 if object 
     is present otherwise 0), 2nd and 3rd dimension would be upper x and y, 4th and 5th would be width and height)+ C (class probability)]
     
    Labels are not of the shape [batch_size, no_grid, no_grid, 5*B+C] because for each bounding box predictor per cell
    dimensions of the object are same
     
    #Dimensions of prediction:
    Prediction will be of dimension [batch_size, no_grid, no_grid, 5*B+C]
    in the 4th dimension: (1st B are confidence score, next 4*B are [x,y,w,h] repeated B times, and last C are class prob)
    """ 
    
    ### extracting classes and boxes from labels
    label_classes = tf.reshape(labels[:,:,:,5:], [batch_size, no_grid, no_grid, no_classes])
    label_boxes = tf.reshape(labels[:,:,:,1:5], [batch_size, no_grid, no_grid, 1, 4])
    label_boxes = tf.tile(label_boxes, [1,1,1,boxes_per_cell,1]) / image_size
    
    ### extracting classes, boxes and confidence from prediction
    predict_classes = tf.reshape(prediction[:,:,:,5*boxes_per_cell:], [batch_size, no_grid, no_grid, no_classes])
    predict_boxes = tf.reshape(prediction[:,:,:,boxes_per_cell:5*boxes_per_cell], [batch_size, no_grid, no_grid, boxes_per_cell, 4])
    predict_confidence = tf.reshape(prediction[:,:,:,:boxes_per_cell], [batch_size, no_grid, no_grid, boxes_per_cell])

    ###Calculating offset for correction of prediction and formatting of labels
    offset = np.transpose(np.reshape(np.array([np.arange(no_grid)] * no_grid * boxes_per_cell), (boxes_per_cell, no_grid, no_grid)), (1, 2, 0))
    offset = tf.constant(offset, dtype = tf.float32)
    offset = tf.reshape(offset, [1, no_grid, no_grid, boxes_per_cell])
    offset = tf.tile(offset, [batch_size, 1, 1, 1])
    
    ###Correction and formatting
    #Correcting predict_boxes for calculating IOU
    corr_predict_boxes = tf.stack([(predict_boxes[:,:,:,:,0] + offset) / no_grid,
                                  (predict_boxes[:,:,:,:,1] + np.transpose(offset, (0,2,1,3))) / no_grid,
                                  tf.square(predict_boxes[:,:,:,:,2]),
                                  tf.square(predict_boxes[:,:,:,:,3])])
    corr_predict_boxes = tf.transpose(corr_predict_boxes, [1,2,3,4,0]) #As tf.stack appends new dimension in the front
    
    #Formatting label boxes according by normalization and subtracting offset
    format_label_boxes = tf.stack([label_boxes[:,:,:,:,0] * no_grid - offset,
                                   label_boxes[:,:,:,:,1] + np.transpose(offset, (0,2,1,3)) / no_grid,
                                   tf.sqrt(label_boxes[:,:,:,:,2]),
                                   tf.sqrt(label_boxes[:,:,:,:,3])])
    format_label_boxes = tf.transpose(format_label_boxes, [1,2,3,4,0]) #As tf.stack appends new dimension in the front
    
    #Calculated IOU for each box (batch_size, no_grid, no_grid, boxes_per_cell)
    calc_iou_boxes = calc_iou(corr_predict_boxes, label_boxes)
    
    ###Calculating difference masks
    #Mask if the object is present in the cell or not (batch_size, no_grid, no_grid, 1)
    object_presence_map = tf.reshape(labels[:,:,:,0], [batch_size, no_grid, no_grid, 1]) #if object is present in the grid cell or not
    
    #Calculating object max (float 1,0 of shape (batch_size, no_grid, no_grid, boxes_per_cell))
    iou_mask = tf.reduce_max(calc_iou_boxes, axis = 3, keepdims = True)
    iou_mask = tf.cast((calc_iou_boxes>=iou_mask), dtype=tf.float64)
    object_mask = iou_mask * object_presence_map
    
    #Calculating no object mask (float 1,0 of shape (batch_size, no_grid, no_grid, boxes_per_cell))
    no_object_mask = tf.ones_like(object_mask, dtype=tf.float64) - object_mask
    
    #Calculating coordinate mask (float 1,0 of shape (batch_size, no_grid, no_grid, boxes_per_cell, 1))
    coord_mask = tf.expand_dims(object_mask, 4)
    
    
    ### Losses
    #Class loss
    class_loss = class_scale * tf.reduce_mean(tf.reduce_sum(tf.square(object_presence_map*(predict_classes - label_classes)), axis = [1,2,3]))
    
    #Calculating confidence loss
    confidence_obj_loss = object_mask * (predict_confidence - calc_iou_boxes)  # we want to make confidence score same as iou when obj is present
    confidence_obj_loss = confidence_obj_scale * tf.reduce_mean(tf.reduce_sum(tf.square(confidence_obj_loss), axis = [1,2,3]))
    
    confidence_noobj_loss = no_object_mask * (predicted_confidence) #we want to make the confidence score 0 when obj is not present
    confidence_noobj_loss = confidence_noobj_scale * tf.reduce_mean(tf.reduce_sum(tf.square(confidence_noobj_loss), axis = [1,2,3]))
    
    #Calculating coordinates loss
    coord_loss = coord_mask * (predict_boxes - format_label_boxes)
    coord_loss = coord_scale * tf.reduce_mean(tf.reduce_sum(tf.square(coord_loss), axis = [1,2,3]))
    
    #Adding up the losses
    total_loss = class_loss + confidence_obj_loss + confidence_noobj_loss + coord_loss
    
#     sess = tf.Session()
    
#     image_placeholder = tf.placeholder("float", (batch_size, 2048, 2048, 3))
    
    

In [44]:
a = tf.constant([[1,2,3,4,5],[3,4,56,7,2]])
print(a.get_shape())
with tf.Session() as sess:
    print(sess.run(a[:,1:4]))

(2, 5)
[[ 2  3  4]
 [ 4 56  7]]


In [38]:
batch_size = 50
cell_size = 8
boxes_per_cell = 5

a1 = tf.random_uniform(np.array([batch_size, cell_size, cell_size, boxes_per_cell, 4]))
a2 = tf.random_uniform(np.array([batch_size, cell_size, cell_size, boxes_per_cell, 4]))

c = calc_iou(a1, a2)
print(c.get_shape())
d = tf.reduce_max(c, 3, keepdims=True)
print(d.get_shape())
# with tf.Session() as sess:
#     print(sess.run(c))

(50, 8, 8, 5)
(50, 8, 8, 1)


In [31]:
def calc_iou(boxes1, boxes2, scope = 'iou'):
    
    with tf.variable_scope(scope):
        boxes1 = tf.stack([boxes1[:, :, :, :, 0] - boxes1[:, :, :, :, 2] / 2.0,
                           boxes1[:, :, :, :, 1] - boxes1[:, :, :, :, 3] / 2.0,
                           boxes1[:, :, :, :, 0] + boxes1[:, :, :, :, 2] / 2.0,
                           boxes1[:, :, :, :, 1] + boxes1[:, :, :, :, 3] / 2.0])
        boxes1 = tf.transpose(boxes1, [1, 2, 3, 4, 0])

        boxes2 = tf.stack([boxes2[:, :, :, :, 0] - boxes2[:, :, :, :, 2] / 2.0,
                           boxes2[:, :, :, :, 1] - boxes2[:, :, :, :, 3] / 2.0,
                           boxes2[:, :, :, :, 0] + boxes2[:, :, :, :, 2] / 2.0,
                           boxes2[:, :, :, :, 1] + boxes2[:, :, :, :, 3] / 2.0])
        boxes2 = tf.transpose(boxes2, [1, 2, 3, 4, 0])

        lu = tf.maximum(boxes1[:, :, :, :, :2], boxes2[:, :, :, :, :2])
        rd = tf.minimum(boxes1[:, :, :, :, 2:], boxes2[:, :, :, :, 2:])

        intersection = tf.maximum(0.0, rd - lu)
        inter_square = intersection[:, :, :, :, 0] * intersection[:, :, :, :, 1]

        square1 = (boxes1[:, :, :, :, 2] - boxes1[:, :, :, :, 0]) * (boxes1[:, :, :, :, 3] - boxes1[:, :, :, :, 1])
        square2 = (boxes2[:, :, :, :, 2] - boxes2[:, :, :, :, 0]) * (boxes2[:, :, :, :, 3] - boxes2[:, :, :, :, 1])

        union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)

    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [ ]:
predict_boxes = tf.reshape(predicts[:, self.boundary2:], [settings.batch_size, self.cell_size, self.cell_size, self.boxes_per_cell, 4])
#predict_boxes shape = (batch_size, cell_size, cell_size, boxes_per_cell, 4)


boxes = tf.reshape(labels[:, :, :, 1:5], [settings.batch_size, self.cell_size, self.cell_size, 1, 4])
boxes = tf.tile(boxes, [1, 1, 1, self.boxes_per_cell, 1]) / self.image_size
#Boxes shape (batch_size, cell_size, cell_size, boxes_per_cell, 4)

offset = tf.constant(self.offset, dtype = tf.float32)
offset = tf.reshape(offset, [1, self.cell_size, self.cell_size, self.boxes_per_cell])
offset = tf.tile(offset, [settings.batch_size, 1, 1, 1])
predict_boxes_tran = tf.stack([(predict_boxes[:, :, :, :, 0] + offset) / self.cell_size,
                               (predict_boxes[:, :, :, :, 1] + tf.transpose(offset, (0, 2, 1, 3))) / self.cell_size,
                               tf.square(predict_boxes[:, :, :, :, 2]),
                               tf.square(predict_boxes[:, :, :, :, 3])])
predict_boxes_tran = tf.transpose(predict_boxes_tran, [1, 2, 3, 4, 0])
#predict_boxes tran = shape (batch_size, cell_size,cell_size, boxes_per_cell, 4) but with offset added and normalized for cell size

iou_predict_truth = self.calc_iou(predict_boxes_tran, boxes)

object_mask = tf.reduce_max(iou_predict_truth, 3, keep_dims=True)
object_mask = tf.cast((iou_predict_truth >= object_mask), tf.float32) * response


In [28]:
a = tf.constant([[1,2],[3,4]])
b = tf.constant([[5,6],[7,8]])
d = tf.constant([[9,10],[11,12]])
c = tf.stack([a,b,d])
print(c.get_shape())
with tf.Session() as sess:
    print(sess.run(c))

(3, 2, 2)
[[[ 1  2]
  [ 3  4]]

 [[ 5  6]
  [ 7  8]]

 [[ 9 10]
  [11 12]]]


In [23]:
a = tf.constant([[1, 2], [3, 4]])
at = tf.transpose(a, [1,0])
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(at))

[[1 2]
 [3 4]]
[[1 3]
 [2 4]]


In [21]:
import tensorflow as tf

vec = tf.constant([1, 2, 3, 4])
multiply = tf.constant([3])

matrix = tf.reshape(tf.tile(vec, multiply), [ multiply[0], tf.shape(vec)[0]])
with tf.Session() as sess:
    print(sess.run([matrix]))

[array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]], dtype=int32)]


In [10]:
S = 7
B = 2
C = 20
batch_size = 50
a = np.random.rand(batch_size, S, S, 5*B+C)
labels = tf.convert_to_tensor(a)
print(labels.get_shape())

# with tf.Session() as sess:
#     print(sess.run(labels))

(50, 7, 7, 30)


In [14]:
np.reshape([np.arange(7)] * 7 * 5, (5, 7, 7))

array([[[0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6]],

       [[0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6]],

       [[0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6]],

       [[0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6]],

       [[0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],
        [0, 1, 2, 3, 4, 5, 6],


In [15]:
offset = np.transpose(np.reshape([np.arange(7)] * 7 * 5, (5, 7, 7)), (1,2,0))

In [16]:
offset = tf.constant(offset, dtype = tf.float32)
offset = tf.reshape(offset, [1, 7, 7, 5])
offset = tf.tile(offset, [50, 1, 1, 1])

In [20]:
print(offset.get_shape())


(50, 7, 7, 5)


In [19]:
with tf.Session() as sess:
    print(sess.run(offset))

[[[[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]

  [[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]

  [[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]

  ...

  [[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]

  [[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]

  [[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]]


 [[[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
   [5. 5. 5. 5. 5.]
   [6. 6. 6. 6. 6.]]

  [[0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1.]
   [2. 2. 2. 2. 2.]
   ...
   [4. 4. 4. 4. 4.]
 

In [ ]:
self.offset = np.transpose(np.reshape(np.array([np.arange(self.cell_size)] * self.cell_size * self.boxes_per_cell), (self.boxes_per_cell, self.cell_size, self.cell_size)), (1, 2, 0))